In [103]:
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from scipy.linalg import toeplitz
from scipy.signal import convolve
from scipy.stats import pearsonr

In [104]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## <font color=cyan> **Construindo o dado de poço**

In [111]:
# depth axis
z = np.arange(0, 1000, 1)
n_z = len(z)

# time axis
t = np.arange(0, 4.0, 0.004)
n_t = len(t)

# reservoir thikness
reservoir = 550

# vp values for each layer
vp_c1 = 2.8 # sandstone
vp_c2 = 2.4
vp_c3 = 3.2 # high porosity sandstone (reservoir)
vp_c4 = 2.6
vp_c5 = 5.5 # limestone

# vs values for each layer
vs_c1 = 1.5 # sandstone
vs_c2 = 1.1
vs_c3 = 1.7 # high porosity sandstone (reservoir)
vs_c4 = 1.2
vs_c5 = 3.0 # limestone

# rho values for each layer
rho_c1 = 2.3 # sandstone
rho_c2 = 2.4
rho_c3 = 2.0 # high porosity sandstone (reservoir)
rho_c4 = 2.3
rho_c5 = 2.7 # limestone

# vp model
vp = np.zeros((n_z))
vp[0:250] = vp_c1
vp[250:500] = vp_c2
vp[500:reservoir] = vp_c3
vp[reservoir:750] = vp_c4
vp[750:1000] = vp_c5

# vs model
vs = np.zeros((n_z))
vs[0:250] = vs_c1
vs[250:500] = vs_c2
vs[500:reservoir] = vs_c3
vs[reservoir:750] = vs_c4
vs[750:1000] = vs_c5

# rho model
rho = np.zeros((n_z))
rho[0:250] = rho_c1
rho[250:500] = rho_c2
rho[500:reservoir] = rho_c3
rho[reservoir:750] = rho_c4
rho[750:1000] = rho_c5

In [112]:
# vp and vs logs plot
fig = make_subplots(rows=1, cols=2, subplot_titles=('Velocity', 'Density'))

fig.add_trace(go.Scatter(x=vp, y=z, mode='lines', name='Seismic trace',
                         line=dict(color='darkblue')), row=1, col=1)
fig.add_trace(go.Scatter(x=rho, y=z, mode='lines', name='Seismic trace',
                         line=dict(color='crimson')), row=1, col=2)

# xaxis properties
fig.update_xaxes(title_text="(m/s)", row=1, col=1,
                 title_font=dict(size=16), tickfont=dict(size=14))
fig.update_xaxes(title_text="(kg/m³)", row=1, col=2,
                 title_font=dict(size=16), tickfont=dict(size=14))

# yaxis properties
fig.update_yaxes(title_text="Time (s)", row=1, col=1, autorange="reversed",
                 title_font=dict(size=16), tickfont=dict(size=14))
fig.update_yaxes(row=1, col=2, autorange="reversed",
                 title_font=dict(size=16), tickfont=dict(size=14))

fig.update_layout(height=500, width=700, showlegend=False, title_text='Well logs',
                  title_font=dict(size=18))

fig.show()

## <font color=orange> **Realizando a substituição de fluidos**

In [113]:
# selecting the reservoir layer
Vp = vp[500:reservoir]   # Velocidade da onda P (m/s) = 3.8
Vs = vs[500:reservoir]   # Velocidade da onda S (m/s) = 1.8
Rho = rho[500:reservoir] # Densidade volumétrica (kg/m^3) = 2.33

# initial reservoir conditions
Por = 0.30      # Porosidade
K_min = 36      # Módulo de volume da rocha (GPa)
Rho_min = 2.65 # Densidade do quartzo (kg/m^3)

# initial reservoir fluids conditions
filtrate_sat = np.array([0.0, 0.05, 0.2, 0.4, 0.6, 0.8, 1.0]) # filtrate saturation fraction
K_hc = 1.15    # oil bulk modulus in GPa
K_filtrate = 3.0    # filtrate bulk modulus in GPa
rho_hc = 0.8  # density of gas in g/cm3
rho_mud = 1.3   # density of water in g/cm3

vp_values = np.zeros((len(Vp), len(filtrate_sat)))
vs_values = np.zeros((len(Vs), len(filtrate_sat)))
rho_values = np.zeros((len(Rho), len(filtrate_sat)))
K_dry_values = np.zeros(len(filtrate_sat))
K_fluid_values = np.zeros(len(filtrate_sat))
Rho_fluid_values = np.zeros(len(filtrate_sat))
K_sat_values = np.zeros(len(filtrate_sat))
Rho_new_values = np.zeros(len(filtrate_sat))
Vs_new_values = np.zeros(len(filtrate_sat))
Vp_new_values = np.zeros(len(filtrate_sat))
Swe_values = []

for i in range(len(filtrate_sat)):
    Swe = filtrate_sat[i]
    Swe_values.append(Swe)
    for j in range(len(Vp)):

        # Shear modulus
        mu = Rho[j] * (Vs[j]**2)

        # dry rock bulk modulus
        K_dry = K_min * (1 - Por)
        K_dry_values[i] = K_dry

        # fluid bulk modulus
        K_fluid = 1 / ((Swe / K_filtrate) + ((1 - Swe) / K_hc))
        K_fluid_values[i] = K_fluid

        # fluid density
        Rho_fluid = (1 - Swe) * rho_hc + Swe * rho_mud
        Rho_fluid_values[i] = Rho_fluid

        # saturated bulk modulus
        K_sat = K_dry +( ((1-(K_dry/K_min))**2)/(Por/K_fluid + (1-Por)/K_min + K_dry/K_min**2))
        K_sat_values[i] = K_sat

        # new density
        Rho_new = Rho_min + Por * Rho_fluid
        Rho_new_values[i] = Rho_new

        # new vs
        Vs_new = np.sqrt(mu / Rho_new)
        Vs_new_values[i] = Vs_new  # Use o índice j para atribuir o valor correto

        # new vp
        Vp_new = np.sqrt(((K_sat + (4/3)*mu)/Rho_new))
        Vp_new_values[i] = Vp_new  # Use o índice j para atribuir o valor correto

        # Updating the results for each saturation
        vp_values[j, i] = Vp_new  # Corrigido para atribuir valor a cada elemento
        vs_values[j, i] = Vs_new  # Corrigido para atribuir valor a cada elemento
        rho_values[j, i] = Rho_new  # Corrigido para atribuir valor a cada elemento

Swe_values = np.array(Swe_values)

vp_new_rsv = np.zeros((len(vp), len(filtrate_sat)))
vs_new_rsv = np.zeros((len(vs), len(filtrate_sat)))
rho_new_rsv = np.zeros((len(rho), len(filtrate_sat)))

for i in range(len(filtrate_sat)):
    vp_new_rsv[500:reservoir, i] = vp_values[:, i]  # Corrigido para atribuir a toda a extensão da faixa
    vs_new_rsv[500:reservoir, i] = vs_values[:, i]  # Corrigido para atribuir a toda a extensão da faixa
    rho_new_rsv[500:reservoir, i] = rho_values[:, i]  # Corrigido para atribuir a toda a extensão da faixa

# new vp log
vp_new_rsv[0:250, :] = vp_c1
vp_new_rsv[250:500, :] = vp_c2
vp_new_rsv[reservoir:750, :] = vp_c4
vp_new_rsv[750:1000, :] = vp_c5

# new vs log
vs_new_rsv[0:250, :] = vs_c1
vs_new_rsv[250:500, :] = vs_c2
vs_new_rsv[reservoir:750, :] = vs_c4
vs_new_rsv[750:1000, :] = vs_c5

# new rho log
rho_new_rsv[0:250, :] = rho_c1
rho_new_rsv[250:500, :] = rho_c2
rho_new_rsv[reservoir:750, :] = rho_c4
rho_new_rsv[750:1000, :] = rho_c5

In [114]:
print(K_dry_values)
print(K_fluid_values)
print(K_sat_values)
print(Rho_fluid_values)
print(Rho_new_values)
print(Vp_new_values)
print(Vs_new_values)

[25.2 25.2 25.2 25.2 25.2 25.2 25.2]
[1.15       1.18658641 1.31178707 1.52654867 1.82539683 2.26973684
 3.        ]
[25.50024174 25.50852033 25.5363423  25.58231069 25.64283337 25.72612256
 25.848     ]
[0.8   0.825 0.9   1.    1.1   1.2   1.3  ]
[2.89   2.8975 2.92   2.95   2.98   3.01   3.04  ]
[3.38973151 3.38576358 3.37410612 3.35922597 3.34531126 3.33275246
 3.32230632]
[1.41421356 1.41238207 1.40693001 1.39975785 1.39269426 1.38573655
 1.37888208]


In [115]:
# import numpy as np

# # Dados fornecidos
# Vp = vp[500:530]   # Velocidade da onda P (m/s)
# Vs = vs[500:530]   # Velocidade da onda S (m/s)
# Rho = rho[500:530] # Densidade volumétrica (kg/m^3)

# # Condições iniciais do reservatório
# Por = 0.30         # Porosidade
# K_min = 38         # Módulo de volume da rocha (GPa)
# Rho_quartz = 2.65  # Densidade do quartzo (kg/m^3)

# # Condições iniciais dos fluidos no reservatório
# Swe_sat = np.array([0, 0.05, 0.2, 0.4, 0.6, 0.8, 1]) # Fracção de saturação de filtrado
# K_gas = 1.15    # Módulo de volume do gás (GPa)
# K_wat = 3.0     # Módulo de volume da água (GPa)
# rho_gas = 0.8   # Densidade do gás (g/cm³)
# rho_wat = 1.3   # Densidade da água (g/cm³)

# # Convertendo densidades de g/cm³ para kg/m³
# rho_gas *= 1000
# rho_wat *= 1000

# # Arrays para armazenar os resultados
# vp_values = np.zeros((len(Vp), len(Swe_sat)))
# vs_values = np.zeros((len(Vs), len(Swe_sat)))
# rho_values = np.zeros((len(Rho), len(Swe_sat)))

# # Inicialização dos vetores para valores calculados
# K_dry_values = np.zeros(len(Swe_sat))
# K_final_fluid_values = np.zeros(len(Swe_sat))
# Rho_final_fluid_values = np.zeros(len(Swe_sat))
# K_final_sat_values = np.zeros(len(Swe_sat))
# Rho_final_values = np.zeros(len(Swe_sat))
# Vs_new_values = np.zeros(len(Swe_sat))
# Vp_new_values = np.zeros(len(Swe_sat))

# for i, Swe in enumerate(Swe_sat):
#     for j in range(len(Vp)):
#         # Módulo de volume inicial da rocha saturada
#         K_init_sat = Rho[j] * (Vp[j] ** 2 - (4 / 3) * (Vs[j] ** 2))

#         # Módulo de cisalhamento (Shear modulus)
#         mu = Rho[j] * Vs[j] ** 2

#         # Módulo de volume da rocha seca
#         K_dry = K_min * (1 - Por)
#         K_dry_values[i] = K_dry

#         # Módulo de volume do fluido final
#         K_final_fluid = (K_wat * K_gas) / (K_gas * (1 - Swe) + K_wat * Swe)
#         K_final_fluid_values[i] = K_final_fluid

#         # Densidade do fluido final
#         Rho_final_fluid = (1 - Swe) * rho_gas + Swe * rho_wat
#         Rho_final_fluid_values[i] = Rho_final_fluid

#         # Módulo de volume da rocha saturada final
#         K_final_sat = K_dry + ((1 - K_dry / K_min) ** 2) / ((Por / K_final_fluid) + ((1 - Por) / K_min) - (K_dry / K_min ** 2))
#         K_final_sat_values[i] = K_final_sat

#         # Densidade final da rocha saturada
#         Rho_final = Rho_quartz * (1 - Por) + Rho_final_fluid * Por
#         Rho_final_values[i] = Rho_final

#         # Nova velocidade da onda S
#         Vs_new = np.sqrt(mu / Rho_final)
#         Vs_new_values[i] = Vs_new

#         # Nova velocidade da onda P
#         Vp_new = np.sqrt((K_final_sat + (4 / 3) * mu) / Rho_final)
#         Vp_new_values[i] = Vp_new

#         # Atualizando os arrays de resultados
#         vp_values[j, i] = Vp_new
#         vs_values[j, i] = Vs_new
#         rho_values[j, i] = Rho_final

# # Novo vetor para armazenar os resultados
# vp_new_rsv = np.zeros((len(vp), len(Swe_sat)))
# vs_new_rsv = np.zeros((len(vs), len(Swe_sat)))
# rho_new_rsv = np.zeros((len(rho), len(Swe_sat)))

# # Atribuindo os valores calculados à faixa do reservatório
# for i in range(len(Swe_sat)):
#     vp_new_rsv[500:530, i] = vp_values[:, i]
#     vs_new_rsv[500:530, i] = vs_values[:, i]
#     rho_new_rsv[500:530, i] = rho_values[:, i]

# # Atualizando os vetores com valores padrão para outras faixas
# # Novas velocidades de onda P
# vp_new_rsv[0:250, :] = 2.26
# vp_new_rsv[250:500, :] = 4.78
# vp_new_rsv[530:750, :] = 5.43
# vp_new_rsv[750:1000, :] = 5.70

# # Novas velocidades de onda S
# vs_new_rsv[0:250, :] = 3.26
# vs_new_rsv[250:500, :] = 3.01
# vs_new_rsv[530:750, :] = 3.47
# vs_new_rsv[750:1000, :] = 3.09

# # Novas densidades
# rho_new_rsv[0:250, :] = 2.59
# rho_new_rsv[250:500, :] = 2.70
# rho_new_rsv[530:750, :] = 2.60
# rho_new_rsv[750:1000, :] = 2.65


## <font color=cyan>**Mineralogia e saturação mistas**

In [116]:
# import numpy as np

# # Selecting the reservoir layer
# Vp = vp[500:530]   # Velocidade da onda P (m/s)
# Vs = vs[500:530]   # Velocidade da onda S (m/s)
# Rho = rho[500:530] # Densidade volumétrica (kg/m^3)

# # Initial reservoir conditions
# Por = 0.3         # Porosidade
# K_min = 38       # Módulo de volume da rocha (GPa)
# Rho_quartz = 2.65 # Densidade do quartzo (kg/m^3)

# # Condições iniciais dos fluidos no reservatório
# Swe_sat = np.array([0, 0.05, 0.2, 0.4, 0.6, 0.8, 1]) # filtrate saturation fraction
# K_gas = 1.15       # gas bulk modulus in GPa
# K_wat = 3.0   # water bulk modulus in GPa
# rho_gas = 0.75  # density of gas in g/cm3
# rho_wat = 2.0  # density of water in g/cm3

# vp_values = np.zeros((len(Vp), len(Swe_sat)))
# vs_values = np.zeros((len(Vs), len(Swe_sat)))
# rho_values = np.zeros((len(Rho), len(Swe_sat)))
# K_dry_values = np.zeros(len(Swe_sat))
# K_final_fluid_values = np.zeros(len(Swe_sat))
# Rho_final_fluid_values = np.zeros(len(Swe_sat))
# K_final_sat_values = np.zeros(len(Swe_sat))
# Rho_final_values = np.zeros(len(Swe_sat))
# Vs_new_values = np.zeros(len(Swe_sat))
# Vp_new_values = np.zeros(len(Swe_sat))

# for i, Swe in enumerate(Swe_sat):
#     for j in range(len(Vp)):  # Adicione este loop para iterar sobre cada elemento de Vp, Vs e Rho
#         # Bulk modulus before substitution
#         K_init_sat = Rho[j] * (Vp[j] ** 2 - (4 / 3) * (Vs[j] ** 2))

#         # Shear modulus
#         mu = Rho[j] * Vs[j] ** 2

#         # K_init_fluid = 1 / ((Swe/K_gas) + ((1-Swe) / K_wat))
#         K_init_fluid = 1 / (((1 - Swe) / K_gas) + (Swe / K_wat))

#         # Rho_init_fluid = Swe*rho_gas + (1-Swe)*rho_wat
#         Rho_init_fluid = (1 - Swe) * rho_gas + Swe * rho_wat

#         K_dry = K_min * (1 - Por)
#         K_dry_values[i] = K_dry

#         K_final_fluid = 1 / (((1 - Swe) / K_gas) + (Swe / K_wat))
#         K_final_fluid_values[i] = K_final_fluid

#         Rho_final_fluid = (1 - Swe) * rho_gas + Swe * rho_wat
#         Rho_final_fluid_values[i] = Rho_final_fluid

#         K_final_sat = K_dry + ((1 - K_dry / K_min) ** 2 / ((Por / K_final_fluid) + ((1 - Por) / K_min) - (K_dry / K_min ** 2)))
#         K_final_sat_values[i] = K_final_sat

#         mu = Rho[j] * Vs[j] ** 2  # shear modulus is unchanged according to Gassmann

#         Rho_final = Rho_quartz * (1 - Por) + Rho_final_fluid * Por
#         Rho_final_values[i] = Rho_final

#         Vs_new = np.sqrt(mu / Rho_final)
#         Vs_new_values[i] = Vs_new  # Use o índice j para atribuir o valor correto

#         Vp_new = np.sqrt((K_final_sat + ((4 / 3) * mu)) / Rho_final)
#         Vp_new_values[i] = Vp_new  # Use o índice j para atribuir o valor correto

#         # Atualizando os arrays de resultados para cada fração de saturação
#         vp_values[j, i] = Vp_new  # Corrigido para atribuir valor a cada elemento
#         vs_values[j, i] = Vs_new  # Corrigido para atribuir valor a cada elemento
#         rho_values[j, i] = Rho_final  # Corrigido para atribuir valor a cada elemento

# # Novo vetor para armazenar os resultados
# vp_new_rsv = np.zeros((len(vp), len(Swe_sat)))
# vs_new_rsv = np.zeros((len(vs), len(Swe_sat)))
# rho_new_rsv = np.zeros((len(rho), len(Swe_sat)))

# for i in range(len(Swe_sat)):
#     vp_new_rsv[500:530, i] = vp_values[:, i]  # Corrigido para atribuir a toda a extensão da faixa
#     vs_new_rsv[500:530, i] = vs_values[:, i]  # Corrigido para atribuir a toda a extensão da faixa
#     rho_new_rsv[500:530, i] = rho_values[:, i]  # Corrigido para atribuir a toda a extensão da faixa

# # Atualizando os vetores
# # New compressional velocity
# vp_new_rsv[0:250, :] = 5.26
# vp_new_rsv[250:500, :] = 4.78
# vp_new_rsv[530:750, :] = 5.43
# vp_new_rsv[750:1000, :] = 5.70

# # New shear velocity
# vs_new_rsv[0:250, :] = 3.26
# vs_new_rsv[250:500, :] = 3.01
# vs_new_rsv[530:750, :] = 3.47
# vs_new_rsv[750:1000, :] = 3.09

# # New densidade
# rho_new_rsv[0:250, :] = 2.59
# rho_new_rsv[250:500, :] = 2.70
# rho_new_rsv[530:750, :] = 2.60
# rho_new_rsv[750:1000, :] = 2.65

In [117]:
K_dry_values = np.array([K_dry_values[i] for i in range(len(filtrate_sat))])
K_final_sat_values = np.array([K_final_sat_values[i] for i in range(len(filtrate_sat))])
Rho_final_fluid_values = np.array([Rho_final_fluid_values[i] for i in range(len(filtrate_sat))])
Rho_final_values = np.array([Rho_final_values[i] for i in range(len(filtrate_sat))])
Vp_new_values = np.array([Vp_new_values[i] for i in range(len(filtrate_sat))])
Vs_new_values = np.array([Vs_new_values[i] for i in range(len(filtrate_sat))])

# creationg subplots
fig = make_subplots(rows=3, cols=2, subplot_titles=('K<sub>dry</sub> vs. saturation', 'K<sub>sat</sub> vs. saturation',
                                                    'Rho<sub>fl</sub> vs. saturation', 'Rho vs. saturation',
                                                    'V<sub>p</sub> vs. saturation', 'V<sub>s</sub> vs. saturation'))

# adding subplots
fig.add_trace(go.Scatter(x=filtrate_sat, y=K_dry_values, mode='lines+markers', name='K_dry', marker=dict(color='red')), row=1, col=1)
fig.add_trace(go.Scatter(x=filtrate_sat, y=K_final_sat_values, mode='lines+markers', name='K_final_sat', marker=dict(color='red')), row=1, col=2)
fig.add_trace(go.Scatter(x=filtrate_sat, y=Rho_final_fluid_values, mode='lines+markers', name='K_dry', marker=dict(color='orange')), row=2, col=1)
fig.add_trace(go.Scatter(x=filtrate_sat, y=Rho_final_values, mode='lines+markers', name='K_dry', marker=dict(color='orange')), row=2, col=2)
fig.add_trace(go.Scatter(x=filtrate_sat, y=Vp_new_values, mode='lines+markers', name='K_dry', marker=dict(color='blue')), row=3, col=1)
fig.add_trace(go.Scatter(x=filtrate_sat, y=Vs_new_values, mode='lines+markers', name='K_dry', marker=dict(color='blue')), row=3, col=2)

# xaxis properties
fig.update_xaxes(title_text='Filtrate saturation (v/v)', row=1, col=1)
fig.update_xaxes(title_text='Filtrate saturation (v/v)', row=1, col=2)
fig.update_xaxes(title_text='Filtrate saturation (v/v)', row=2, col=1)
fig.update_xaxes(title_text='Filtrate saturation (v/v)', row=2, col=2)
fig.update_xaxes(title_text='Filtrate saturation (v/v)', row=3, col=1)
fig.update_xaxes(title_text='Filtrate saturation (v/v)', row=3, col=2)

# yaxis properties
fig.update_yaxes(title_text='K<sub>dry</sub> (GPa)', row=1, col=1)
fig.update_yaxes(title_text='K<sub>sat</sub> (GPa)', row=1, col=2)
fig.update_yaxes(title_text='Rho<sub>fl</sub> (g/cm3)', row=2, col=1)
fig.update_yaxes(title_text='Rho (g/cm3)', row=2, col=2)
fig.update_yaxes(title_text='V<sub>p</sub> (km/s)', row=3, col=1)
fig.update_yaxes(title_text='V<sub>s</sub> (km/s)', row=3, col=2)

# figure layout
fig.update_layout(height=700, width=800, showlegend=False, title_text='Parametes - 30% of porosity', title_font=dict(size=18))

# show figure
fig.show()

In [118]:
# differents saturations plot
fig = make_subplots(rows=1, cols=3, subplot_titles=('Compressional', 'Density', 'Shear'))

fig.add_trace(go.Scatter(x=vp_new_rsv[:,0], y=z, mode='lines', name='Sw=80%',
                         line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=vp_new_rsv[:,1], y=z, mode='lines', name='Sw=60%',
                         line=dict(color='green')), row=1, col=1)
fig.add_trace(go.Scatter(x=vp_new_rsv[:,2], y=z, mode='lines', name='Sw=40%',
                         line=dict(color='violet')), row=1, col=1)
fig.add_trace(go.Scatter(x=vp_new_rsv[:,3], y=z, mode='lines', name='Sw=20%',
                         line=dict(color='red')), row=1, col=1)
fig.add_trace(go.Scatter(x=vp_new_rsv[:,4], y=z, mode='lines', name='Sw=5%',
                         line=dict(color='yellow')), row=1, col=1)
fig.add_trace(go.Scatter(x=vp_new_rsv[:,5], y=z, mode='lines', name='Sw=0%',
                         line=dict(color='black')), row=1, col=1)

fig.add_trace(go.Scatter(x=rho_new_rsv[:,0], y=z, name='Sw=80%',
                         line=dict(color='blue')), row=1, col=2)
fig.add_trace(go.Scatter(x=rho_new_rsv[:,1], y=z, name='Sw=60%',
                         line=dict(color='green')), row=1, col=2)
fig.add_trace(go.Scatter(x=rho_new_rsv[:,2], y=z, name='Sw=40%',
                         line=dict(color='violet')), row=1, col=2)
fig.add_trace(go.Scatter(x=rho_new_rsv[:,3], y=z, name='Sw=20%',
                         line=dict(color='red')), row=1, col=2)
fig.add_trace(go.Scatter(x=rho_new_rsv[:,4], y=z, mode='lines', name='Sw=5%',
                         line=dict(color='yellow')), row=1, col=2)
fig.add_trace(go.Scatter(x=rho_new_rsv[:,5], y=z, mode='lines', name='Sw=0%',
                         line=dict(color='black')), row=1, col=2)

fig.add_trace(go.Scatter(x=vs_new_rsv[:,0], y=z, name='Sw=80%',
                         line=dict(color='blue')), row=1, col=3)
fig.add_trace(go.Scatter(x=vs_new_rsv[:,1], y=z, name='Sw=80%',
                         line=dict(color='green')), row=1, col=3)
fig.add_trace(go.Scatter(x=vs_new_rsv[:,2], y=z, name='Sw=80%',
                         line=dict(color='violet')), row=1, col=3)
fig.add_trace(go.Scatter(x=vs_new_rsv[:,3], y=z, name='Sw=80%',
                         line=dict(color='red')), row=1, col=3)
fig.add_trace(go.Scatter(x=vs_new_rsv[:,4], y=z, mode='lines', name='Sw=5%',
                         line=dict(color='yellow')), row=1, col=3)
fig.add_trace(go.Scatter(x=vs_new_rsv[:,5], y=z, mode='lines', name='Sw=0%',
                         line=dict(color='black')), row=1, col=3)

# xaxis properties
fig.update_xaxes(title_text="(m/s)", row=1, col=1,
                 title_font=dict(size=16), tickfont=dict(size=14))
fig.update_xaxes(title_text="(kg/m³)", row=1, col=2,
                 title_font=dict(size=16), tickfont=dict(size=14))
fig.update_xaxes(title_text="(m/s)", row=1, col=3,
                 title_font=dict(size=16), tickfont=dict(size=14))

# yaxis properties
fig.update_yaxes(title_text="Depth (m)", row=1, col=1, autorange="reversed",
                 title_font=dict(size=16), tickfont=dict(size=14))
fig.update_yaxes(autorange="reversed", row=1, col=2,
                 title_font=dict(size=16), tickfont=dict(size=14))
fig.update_yaxes(autorange="reversed", row=1, col=3,
                 title_font=dict(size=16), tickfont=dict(size=14))

fig.update_layout(height=700, showlegend=False, width=900, title_text='Saturations on reservoir',
                  title_font=dict(size=18))

fig.show()

## <font color=orange> **Well tie**

In [119]:
# impedance
ip = np.zeros((len(t), len(filtrate_sat)))
for i in range(len(filtrate_sat)):
    for j in range(len(t)):
        ip[j, i] = vp_new_rsv[j, i] * rho_new_rsv[j, i]

# reflectivity
R = np.zeros((len(t) - 1, len(filtrate_sat)))
for i in range(len(filtrate_sat)):
    for j in range(len(t) - 1):
        R[j, i] = (ip[j + 1, i] - ip[j, i]) / (ip[j + 1, i] + ip[j, i])

In [120]:
# seismic trace and wavelet from seismic model
trace_model = np.loadtxt('/content/drive/MyDrive/doutorado/dados/trace_50_30_20.txt')
wavelet_model = np.loadtxt('/content/drive/MyDrive/doutorado/dados/wav_20.txt')

In [121]:
# convolution
synt = np.zeros((len(t), len(filtrate_sat)))
for i in range(len(filtrate_sat)):
    sp = np.convolve(R[:, i], wavelet_model)
    synt[:, i] = sp[:len(t)]

In [122]:
# synthetic trace
fig = make_subplots(rows=1, cols=5, subplot_titles=('Impedance', 'Reflectivity', 'Wavelet', 'Synthetic 1', 'Seismic trace'))

fig.add_trace(go.Scatter(x=ip[:,0], y=t, name='Sw=100%',
                         line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=ip[:,1], y=t, name='Sw=60%',
                         line=dict(color='green')), row=1, col=1)
fig.add_trace(go.Scatter(x=ip[:,2], y=t, name='Sw=40%',
                         line=dict(color='violet')), row=1, col=1)
fig.add_trace(go.Scatter(x=ip[:,3], y=t, name='Sw=20%',
                         line=dict(color='red')), row=1, col=1)
fig.add_trace(go.Scatter(x=ip[:,4], y=t, mode='lines', name='Sw=5%',
                         line=dict(color='yellow')), row=1, col=1)
fig.add_trace(go.Scatter(x=ip[:,5], y=t, mode='lines', name='Sw=0%',
                         line=dict(color='black')), row=1, col=1)

fig.add_trace(go.Scatter(x=R[:,0], y=t, name='Sw=80%',
                         line=dict(color='blue')), row=1, col=2)
fig.add_trace(go.Scatter(x=R[:,1], y=t, name='Sw=60%',
                         line=dict(color='green')), row=1, col=2)
fig.add_trace(go.Scatter(x=R[:,2], y=t, name='Sw=40%',
                         line=dict(color='violet')), row=1, col=2)
fig.add_trace(go.Scatter(x=R[:,3], y=t, name='Sw=20%',
                         line=dict(color='red')), row=1, col=2)
fig.add_trace(go.Scatter(x=R[:,4], y=t, mode='lines', name='Sw=5%',
                         line=dict(color='yellow')), row=1, col=2)
fig.add_trace(go.Scatter(x=R[:,5], y=t, mode='lines', name='Sw=0%',
                         line=dict(color='black')), row=1, col=2)

fig.add_trace(go.Scatter(x=wavelet_model, y=t, name='Sw=80%',
                         line=dict(color='blue')), row=1, col=3)

fig.add_trace(go.Scatter(x=synt[:,0], y=t, name='Sw=80%',
                         line=dict(color='blue')), row=1, col=4)
fig.add_trace(go.Scatter(x=synt[:,1], y=t, name='Sw=60%',
                         line=dict(color='green')), row=1, col=4)
fig.add_trace(go.Scatter(x=synt[:,2], y=t, name='Sw=40%',
                         line=dict(color='violet')), row=1, col=4)
fig.add_trace(go.Scatter(x=synt[:,3], y=t, name='Sw=20%',
                         line=dict(color='red')), row=1, col=4)
fig.add_trace(go.Scatter(x=synt[:,4], y=t, mode='lines', name='Sw=5%',
                         line=dict(color='yellow')), row=1, col=4)
fig.add_trace(go.Scatter(x=synt[:,5], y=t, mode='lines', name='Sw=0%',
                         line=dict(color='black')), row=1, col=4)

fig.add_trace(go.Scatter(x=trace_model, y=t, name='Seismic model',
                         line=dict(color='blue')), row=1, col=5)

# Update xaxis properties
fig.update_xaxes(title_text='kg/(m²s)', row=1, col=1,
                 title_font=dict(size=16), tickfont=dict(size=14))
fig.update_xaxes(title_text='Unitless', row=1, col=2,
                 title_font=dict(size=16), tickfont=dict(size=14))
fig.update_xaxes(title_text='Amplitude (a.u.)', row=1, col=3,
                 title_font=dict(size=16), tickfont=dict(size=14))
fig.update_xaxes(title_text='Amplitude (a.u.)', row=1, col=4,
                 title_font=dict(size=16), tickfont=dict(size=14))
fig.update_xaxes(title_text='Amplitude (a.u.)', row=1, col=5,
                 title_font=dict(size=16), tickfont=dict(size=14))

# Update yaxis properties
fig.update_yaxes(title_text="Time (s)", row=1, col=1, autorange="reversed",
                 title_font=dict(size=16), tickfont=dict(size=14))
fig.update_yaxes(autorange="reversed", row=1, col=2,
                 title_font=dict(size=16), tickfont=dict(size=14))
fig.update_yaxes(autorange="reversed", row=1, col=3,
                 title_font=dict(size=16), tickfont=dict(size=14))
fig.update_yaxes(autorange="reversed", row=1, col=4,
                 title_font=dict(size=16), tickfont=dict(size=14))
fig.update_yaxes(autorange="reversed", row=1, col=5,
                 title_font=dict(size=16), tickfont=dict(size=14))

fig.update_layout(height=700, width=1200, showlegend=False, title_text='Synthetic generation',
                  title_font=dict(size=16))

fig.show()

## <font color=orange> **Sparse-spike deconvolution**

In [123]:
def conjgrad(A, b, x=None):
    n = len(b)
    if x is None:
        x = np.zeros(n)
    r = b - A @ x
    p = r.copy()
    rsold = np.dot(r, r)

    for _ in range(n):
        Ap = A @ p
        alpha = rsold / np.dot(p, Ap)
        x = x + alpha * p
        r = r - alpha * Ap
        rsnew = np.dot(r, r)
        if np.sqrt(rsnew) < 1e-10:
            break
        p = r + (rsnew / rsold) * p
        rsold = rsnew

    return x

In [124]:
# Sparse-spike deconvolution function
def sparse_decon_l2(d, r, mu, iter_max):
    nr = len(r)
    nt, ntraces = d.shape

    # Increasing zeros to allow full convolution
    d_padded = np.vstack([d, np.zeros((nr - 1, ntraces))])

    # Creating convolution matrix
    R = toeplitz(np.concatenate([r, np.zeros(nt - 1)]), np.concatenate([[r[0]], np.zeros(nt - 1)]))

    # calculating the autocorrelation matrix
    R1 = R.T @ R

    # Creating the regularization matrix
    Q = mu * np.eye(nt)

    # scale value
    sc = 0.0001

    wave = np.zeros((nt, ntraces))

    for itrace in range(ntraces):
        s = d_padded[:, itrace]
        w = np.zeros(nt)

        for k in range(iter_max):
            g = R.T @ s
            Matrix = R1 + Q
            w = conjgrad(Matrix, g, w)

        wave[:, itrace] = w

    dp = np.apply_along_axis(lambda x: convolve(x, r, mode='full')[:nt], axis=0, arr=wave)

    return wave, dp

In [125]:
# estimating the wavelet - using synthetic trace and reflectivity
alpha = 0.01
synt2 = np.zeros((len(t), len(filtrate_sat)))
corrs = []
corrs2 = []
wavelet = []

for i in range(len(filtrate_sat)):
    w, _ = sparse_decon_l2(synt[:, i:i+1], R[:,i], alpha, 5)
    wavelet.append(w.squeeze())  # remove unnecessary dimension

    # synthetic trace 2 from the estimated wavelet
    synt2[:, i] = convolve(R[:,i], wavelet[i], mode='full')[:len(t)]
    synt2[:, i] = synt2[:, i] / np.max(np.abs(synt2[:, i]))
    synt[:, i] = synt[:, i] / np.max(np.abs(synt[:, i]))
    wavelet[i] = wavelet[i] / np.max(np.abs(wavelet[i]))

    corr = np.corrcoef(synt[:, i], synt2[:, i])[0, 1]
    corrs.append(corr)

    corr2 = np.corrcoef(trace_model, synt2[:, i])[0, 1]
    corrs2.append(corr)

# convert list of numpy arrays to a single numpy array
wavelet = np.column_stack(wavelet)

In [126]:
# plot
fig = make_subplots(rows=1, cols=4, subplot_titles=('Reflectivities', 'Synthetic 1', 'Wavelets SS', 'Synthetic 2'))

fig.add_trace(go.Scatter(x=R[:,0], y=t, name='Sw=100%',
                         line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=R[:,1], y=t, name='Sw=60%',
                         line=dict(color='green')), row=1, col=1)
fig.add_trace(go.Scatter(x=R[:,2], y=t, name='Sw=40%',
                         line=dict(color='violet')), row=1, col=1)
fig.add_trace(go.Scatter(x=R[:,3], y=t, name='Sw=20%',
                         line=dict(color='red')), row=1, col=1)
fig.add_trace(go.Scatter(x=R[:,4], y=t, mode='lines', name='Sw=5%',
                         line=dict(color='yellow')), row=1, col=1)
fig.add_trace(go.Scatter(x=R[:,5], y=t, mode='lines', name='Sw=0%',
                         line=dict(color='black')), row=1, col=1)

fig.add_trace(go.Scatter(x=synt[:,0], y=t, name='Sw=100%',
                         line=dict(color='blue')), row=1, col=2)
fig.add_trace(go.Scatter(x=synt[:,1], y=t, name='Sw=60%',
                         line=dict(color='green')), row=1, col=2)
fig.add_trace(go.Scatter(x=synt[:,2], y=t, name='Sw=40%',
                         line=dict(color='violet')), row=1, col=2)
fig.add_trace(go.Scatter(x=synt[:,3], y=t, name='Sw=20%',
                         line=dict(color='red')), row=1, col=2)
fig.add_trace(go.Scatter(x=synt[:,4], y=t, mode='lines', name='Sw=5%',
                         line=dict(color='yellow')), row=1, col=2)
fig.add_trace(go.Scatter(x=synt[:,5], y=t, mode='lines', name='Sw=0%',
                         line=dict(color='black')), row=1, col=2)

fig.add_trace(go.Scatter(x=wavelet[:, 0], y=t, name='Sw=100%',
                         line=dict(color='blue')), row=1, col=3)
fig.add_trace(go.Scatter(x=wavelet[:, 1], y=t, name='Sw=60%',
                         line=dict(color='green')), row=1, col=3)
fig.add_trace(go.Scatter(x=wavelet[:, 2], y=t, name='Sw=40%',
                         line=dict(color='violet')), row=1, col=3)
fig.add_trace(go.Scatter(x=wavelet[:, 3], y=t, name='Sw=20%',
                         line=dict(color='red')), row=1, col=3)
fig.add_trace(go.Scatter(x=wavelet[:, 4], y=t, name='Sw=5%',
                         line=dict(color='yellow')), row=1, col=3)
fig.add_trace(go.Scatter(x=wavelet[:, 5], y=t, name='Sw=0%',
                         line=dict(color='black')), row=1, col=3)

fig.add_trace(go.Scatter(x=synt2[:,0], y=t, name='Sw=80%',
                         line=dict(color='blue')), row=1, col=4)
fig.add_trace(go.Scatter(x=synt2[:,1], y=t, name='Sw=60%',
                         line=dict(color='green')), row=1, col=4)
fig.add_trace(go.Scatter(x=synt2[:,2], y=t, name='Sw=40%',
                         line=dict(color='violet')), row=1, col=4)
fig.add_trace(go.Scatter(x=synt2[:,3], y=t, name='Sw=20%',
                         line=dict(color='red')), row=1, col=4)
fig.add_trace(go.Scatter(x=synt2[:,4], y=t, mode='lines', name='Sw=5%',
                         line=dict(color='yellow')), row=1, col=4)
fig.add_trace(go.Scatter(x=synt2[:,5], y=t, mode='lines', name='Sw=0%',
                         line=dict(color='black')), row=1, col=4)

# xaxis properties
fig.update_xaxes(title_text='Unitless', row=1, col=1,
                 title_font=dict(size=16), tickfont=dict(size=14))
fig.update_xaxes(title_text='Amplitude (a.u.)', row=1, col=2,
                 title_font=dict(size=16), tickfont=dict(size=14))
fig.update_xaxes(title_text='Amplitude (a.u.)', row=1, col=3,
                 title_font=dict(size=16), tickfont=dict(size=14))
fig.update_xaxes(title_text='Amplitude (a.u.)', row=1, col=4,
                 title_font=dict(size=16), tickfont=dict(size=14))

# yaxis properties
fig.update_yaxes(title_text="Time (s)", row=1, col=1, autorange="reversed",
                 title_font=dict(size=16), tickfont=dict(size=14))
fig.update_yaxes(autorange="reversed", row=1, col=2,
                 title_font=dict(size=16), tickfont=dict(size=14))
fig.update_yaxes(autorange="reversed", row=1, col=3,
                 title_font=dict(size=16), tickfont=dict(size=14))
fig.update_yaxes(autorange="reversed", row=1, col=4,
                 title_font=dict(size=16), tickfont=dict(size=14))

fig.update_layout(height=700, width=1200, showlegend=False, title_text='Second synthetic generation ',
                  title_font=dict(size=16))

fig.show()

In [127]:
# correlation plots
fig = make_subplots(rows=6, cols=1,
                    subplot_titles=('0% of invasion', '5% of invasion', '20% of invasion',
                                    '40% of invasion', '80% of invasion', '100% of invasion'))

fig.add_trace(go.Scatter(x=t, y=trace_model, name='Trace model',
                         line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=t, y=synt2[:,0], name='Sw=100%',
                         line=dict(color='red')), row=1, col=1)

fig.add_trace(go.Scatter(x=t, y=trace_model, name='Trace model',
                         line=dict(color='blue')), row=2, col=1)
fig.add_trace(go.Scatter(x=t, y=synt2[:,1], name='Sw=60%',
                         line=dict(color='red')), row=2, col=1)

fig.add_trace(go.Scatter(x=t, y=trace_model, name='Trace model',
                         line=dict(color='blue')), row=3, col=1)
fig.add_trace(go.Scatter(x=t, y=synt2[:,2], name='Sw=40%',
                         line=dict(color='red')), row=3, col=1)

fig.add_trace(go.Scatter(x=t, y=trace_model, name='Trace model',
                         line=dict(color='blue')), row=4, col=1)
fig.add_trace(go.Scatter(x=t, y=synt2[:,3], name='Sw=20%',
                         line=dict(color='red')), row=4, col=1)

fig.add_trace(go.Scatter(x=t, y=trace_model, name='Trace model',
                         line=dict(color='blue')), row=5, col=1)
fig.add_trace(go.Scatter(x=t, y=synt2[:,4], mode='lines', name='Sw=5%',
                         line=dict(color='red')), row=5, col=1)

fig.add_trace(go.Scatter(x=t, y=trace_model, name='Trace model',
                         line=dict(color='blue')), row=6, col=1)
fig.add_trace(go.Scatter(x=t, y=synt2[:,5], mode='lines', name='Sw=0%',
                         line=dict(color='red')), row=6, col=1)

# xaxis properties
fig.update_xaxes(row=1, col=1,
                 title_font=dict(size=16), tickfont=dict(size=14))
fig.update_xaxes(row=2, col=1,
                 title_font=dict(size=16), tickfont=dict(size=14))
fig.update_xaxes(row=3, col=1,
                 title_font=dict(size=16), tickfont=dict(size=14))
fig.update_xaxes(row=4, col=1,
                 title_font=dict(size=16), tickfont=dict(size=14))
fig.update_xaxes(row=5, col=1,
                 title_font=dict(size=16), tickfont=dict(size=14))
fig.update_xaxes(title_text='Time (s)', row=6, col=1,
                 title_font=dict(size=16), tickfont=dict(size=14))


# yaxis properties
fig.update_yaxes(title_text="Amp. (a.u.)", autorange="reversed", row=1, col=1,
                 title_font=dict(size=16), tickfont=dict(size=14))
fig.update_yaxes(title_text="Amp. (a.u.)", autorange="reversed", row=2, col=1,
                 title_font=dict(size=16), tickfont=dict(size=14))
fig.update_yaxes(title_text="Amp. (a.u.)", autorange="reversed", row=3, col=1,
                 title_font=dict(size=16), tickfont=dict(size=14))
fig.update_yaxes(title_text="Amp. (a.u.)", autorange="reversed", row=4, col=1,
                 title_font=dict(size=16), tickfont=dict(size=14))
fig.update_yaxes(title_text="Amp. (a.u.)", autorange="reversed", row=5, col=1,
                 title_font=dict(size=16), tickfont=dict(size=14))
fig.update_yaxes(title_text="Amp. (a.u.)", autorange="reversed", row=6, col=1,
                 title_font=dict(size=16), tickfont=dict(size=14))

# adds the correlation value as an annotation to the subplot
fig.add_annotation(x=0.5, y=0.95, xref='paper', yref='paper',
                   text=f'Pearson correlation coefficient: {corrs2[0]:.6f}', showarrow=False, row=1, col=1, font=dict(size=14))
fig.add_annotation(x=0.5, y=0.95, xref='paper', yref='paper',
                   text=f'Pearson correlation coefficient: {corrs2[1]:.6f}', showarrow=False, row=2, col=1, font=dict(size=14))
fig.add_annotation(x=0.5, y=0.95, xref='paper', yref='paper',
                   text=f'Pearson correlation coefficient: {corrs2[2]:.6f}', showarrow=False, row=3, col=1, font=dict(size=14))
fig.add_annotation(x=0.5, y=0.95, xref='paper', yref='paper',
                   text=f'Pearson correlation coefficient: {corrs2[3]:.6f}', showarrow=False, row=4, col=1, font=dict(size=14))
fig.add_annotation(x=0.5, y=0.95, xref='paper', yref='paper',
                   text=f'Pearson correlation coefficient: {corrs2[4]:.6f}', showarrow=False, row=5, col=1, font=dict(size=14))
fig.add_annotation(x=0.5, y=0.95, xref='paper', yref='paper',
                   text=f'Pearson correlation coefficient: {corrs2[5]:.6f}', showarrow=False, row=6, col=1, font=dict(size=14))

fig.update_layout(height=900, width=900, showlegend=False, title_text='Pearson correlation coefficient - Thickness: 30m, Porosity:35%, Frequency: 20Hz',
                  title_font=dict(size=16))

fig.show()

In [128]:
# saturations values
sat = [0, 5, 20, 40, 60, 100]

fig_corr = go.Figure()

fig_corr.add_trace(go.Scatter(x=sat, y=corrs2, mode='lines+markers', name='Correlation',
                              line=dict(color='blue'), marker=dict(size=8)))

# axis properties
fig.update_xaxes(title_text='Mud filtrate saturation (%)',
                 title_font=dict(size=16), tickfont=dict(size=14))
fig.update_yaxes(title_text='Pearson Correlation Coefficient',
                 title_font=dict(size=16), tickfont=dict(size=14))

# figure layout
fig_corr.update_layout(title='Pearson Correlation Coefficient vs. Mud Filtrate Saturation - 30% of porosity',
                       xaxis_title='Mud filtrate saturation (%)',
                       yaxis_title='Pearson Correlation Coefficient',
                       title_font=dict(size=16),
                       xaxis=dict(tickfont=dict(size=14)),
                       yaxis=dict(tickfont=dict(size=14)),
                       height=500,
                       width=700,
                       title_x=0.5)

fig_corr.show()